In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from nltk.translate.bleu_score import corpus_bleu
from nltk.tokenize import word_tokenize
import nltk


c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\ASUS\AppData\Local\Programs\Python\Python310\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:

nltk.download('punkt')

# Load dataset
dset = load_dataset("SEACrowd/liputan6", trust_remote_code=True)

# Use subsets of the dataset
train_data = dset["train"].select(range(10000))
val_data = dset["validation"].select(range(2000))
test_data = dset["test"].select(range(2000))

# Load tokenizer and model
model_name = "google/mt5-small"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:

# Preprocessing function
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize dataset
tokenized_train = train_data.map(preprocess_function, batched=True, remove_columns=train_data.column_names)
tokenized_val = val_data.map(preprocess_function, batched=True, remove_columns=val_data.column_names)
tokenized_test = test_data.map(preprocess_function, batched=True, remove_columns=test_data.column_names)


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 2000/2000 [00:02<00:00, 789.70 examples/s]


In [6]:

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_29392\3100007604.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [7]:

# Fine-tune the model
trainer.train()


                                                   
  0%|          | 6/7500 [02:29<17:32:26,  8.43s/it]

{'loss': 41.2839, 'grad_norm': 1699.947021484375, 'learning_rate': 1.9920000000000002e-05, 'epoch': 0.0}


                                                   
  0%|          | 6/7500 [03:48<17:32:26,  8.43s/it]

{'loss': 40.8443, 'grad_norm': 2572.601806640625, 'learning_rate': 1.9840000000000003e-05, 'epoch': 0.01}


                                                   
  0%|          | 6/7500 [05:06<17:32:26,  8.43s/it]

...





















































































































































































































































































































































































































































































































                                                     
                                                  

  0%|          | 6/7500 [6:46:16<17:32:26,  8.43s/it]



{'eval_loss': 5.535392761230469, 'eval_runtime': 736.839, 'eval_samples_per_second': 2.714, 'eval_steps_per_second': 0.679, 'epoch': 1.0}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].
                                                     
100%|██████████| 2500/2500 [6:45:18<00:00,  9.73s/it]


{'train_runtime': 24319.1993, 'train_samples_per_second': 0.411, 'train_steps_per_second': 0.103, 'train_loss': 11.98397053527832, 'epoch': 1.0}


TrainOutput(global_step=2500, training_loss=11.98397053527832, metrics={'train_runtime': 24319.1993, 'train_samples_per_second': 0.411, 'train_steps_per_second': 0.103, 'total_flos': 5287496908800000.0, 'train_loss': 11.98397053527832, 'epoch': 1.0})

In [8]:
from nltk.translate.bleu_score import sentence_bleu

# Evaluate BLEU score
def compute_bleu(data):
    references = [[word_tokenize(summary)] for summary in data["summary"]]
    predictions = []
    for article in data["document"]:
        inputs = tokenizer("summarize: " + article, return_tensors="pt", max_length=512, truncation=True)
        outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
        pred_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predictions.append(word_tokenize(pred_summary))
    return corpus_bleu(references, predictions)

average_bleu = compute_bleu(test_data)
print(f"Average BLEU Score: {average_bleu}")

# Predict on test data and print BLEU score for each sample
for i in range(10):  # Predict and display summaries for first 10 examples
    article = test_data[i]["document"]
    reference_summary = test_data[i]["summary"]

    # Generate prediction
    inputs = tokenizer("summarize: " + article, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
    pred_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Tokenize references and prediction
    reference_tokens = word_tokenize(reference_summary)
    prediction_tokens = word_tokenize(pred_summary)

    # Compute BLEU score for the sample
    sample_bleu = sentence_bleu([reference_tokens], prediction_tokens)

    print(f"Article: {article}...")
    print(f"Reference Summary: {reference_summary}")
    print(f"Predicted Summary: {pred_summary}")
    print(f"BLEU Score: {sample_bleu}\n")


Average BLEU Score: 0.0006279388331626488


c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower o

Article: Liputan6 . com , Bangka : Kapal patroli Angkatan Laut Republik Indonesia , Belinyu , baru-baru ini , menangkap tiga kapal nelayan berbendera Thailand , yakni KM Binatama , KM Sumber Jaya II , dan KM Mataram di Perairan Belitung Utara . Ketiga kapal itu ditangkap karena melanggar zona ekonomi ekslusif Indonesia . Saat ini , kapal-kapal itu diamankan di Pos Lanal Pelabuhan Pangkalan Balam , Bangka-Belitung . Menurut Komandan Pangkalan TNI AL Bangka Letnan Kolonel Laut Fredy Egam , selain menangkap tiga kapal , ALRI juga memeriksa 43 anak buah kapal . Mereka disergap saat sedang mengangkat jaring pukat harimau di Perairan Belitung Utara . Dari jumlah itu , hanya enam orang yang dijadikan tersangka , yakni tiga nahkoda dan tiga kepala kamar mesin kapal . Sedangkan ABK yang lain akan dideportasi ke negara asalnya . Meski berhasil menahan enam tersangka , TNI AL gagal mengamankan ikan tangkapan nelayan Thailand tersebut . Sebab , sebelum patroli datang , mereka telah memindahkan pul

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

# Evaluate BLEU score
def compute_bleu(data):
    references = [[word_tokenize(summary)] for summary in data["summary"]]
    predictions = []
    for article in data["document"]:
        inputs = tokenizer("summarize: " + article, return_tensors="pt", max_length=512, truncation=True)
        outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
        pred_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predictions.append(word_tokenize(pred_summary))
    return corpus_bleu(references, predictions)

average_bleu = compute_bleu(test_data)
print(f"Average BLEU Score: {average_bleu}")

# Predict on test data and print BLEU score for each sample
for i in range(10):  # Predict and display summaries for first 10 examples
    article = test_data[i]["document"]
    reference_summary = test_data[i]["summary"]

    # Generate prediction
    inputs = tokenizer("summarize: " + article, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
    pred_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Tokenize references and prediction
    reference_tokens = word_tokenize(reference_summary)
    prediction_tokens = word_tokenize(pred_summary)

    # Compute BLEU score for the sample
    sample_bleu = sentence_bleu([reference_tokens], prediction_tokens)

    print(f"Article: {article}...")
    print(f"Reference Summary: {reference_summary}")
    print(f"Predicted Summary: {pred_summary}")
    print(f"BLEU Score: {sample_bleu}\n")


Average BLEU Score: 0.0004225577576027379
Article: Liputan6 . com , Bangka : Kapal patroli Angkatan Laut Republik Indonesia , Belinyu , baru-baru ini , menangkap tiga kapal nelayan berbendera Thailand , yakni KM Binatama , KM Sumber Jaya II , dan KM Mataram di Perairan Belitung Utara . Ketiga kapal itu ditangkap karena melanggar zona ekonomi ekslusif Indonesia . Saat ini , kapal-kapal itu diamankan di Pos Lanal Pelabuhan Pangkalan Balam , Bangka-Belitung . Menurut Komandan Pangkalan TNI AL Bangka Letnan Kolonel Laut Fredy Egam , selain menangkap tiga kapal , ALRI juga memeriksa 43 anak buah kapal . Mereka disergap saat sedang mengangkat jaring pukat harimau di Perairan Belitung Utara . Dari jumlah itu , hanya enam orang yang dijadikan tersangka , yakni tiga nahkoda dan tiga kepala kamar mesin kapal . Sedangkan ABK yang lain akan dideportasi ke negara asalnya . Meski berhasil menahan enam tersangka , TNI AL gagal mengamankan ikan tangkapan nelayan Thailand tersebut . Sebab , sebelum pat